#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys
import time

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import *
from src.annotator.sem_tagger import *

#### **2. 读取语料**

读取 TSV 格式的平行语料库

In [2]:
# 指定语料库路径
data_file = '../data/raw/test_data.tsv'

# 读取 TSV 格式语料
# 数据存储于 DataFrame 对象
data = load_data(data_file)
print(f'成功读取 {len(data)} 条平行句对')

# 预览数据
# 第一列：汉语原文，选自《鹿鼎记》
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）
print('数据前 5 行如下：')
data.head()

成功读取 10 条平行句对
数据前 5 行如下：


,source,target
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th..."
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...
5,囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！,"One of the soldiers marching alongside, irrita..."


#### **3. 大模型中文语义标注：USAS 赋码集**

In [3]:
# === 加载模型：LLM USAS 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：USAS
# https://ucrel.lancs.ac.uk/usas/
# https://ucrel.lancs.ac.uk/usas/usas_guide.pdf
tagset = 'usas'

# 标注模式：LLM API
mode = 'llm'

# 大模型：deepseek-v3.2 | kimi-k2.5 | glm-4.7 | qwen3-max
llm_model = 'deepseek-v3.2'

# 思考模式：关闭
enable_thinking = False

# 登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/config 文件中设置 LLM_API_KEY=sk-********
t0 = time.time()
llm_zh_usas_tagger = SEMTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
    llm_model=llm_model,
    enable_thinking=enable_thinking,
)
t1 = time.time()
print('LLM USAS 中文语义标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

LLM USAS 中文语义标注模型加载完毕！
加载耗时：1.78s


In [4]:
# === 标注原文：LLM USAS 赋码集 ===

# 注意：
# 为节省 API 调用成本
# 大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，再次运行程序，时间显示为 0.0 秒
llm_zh_usas_annos = annotate_data(
    data['source'],
    llm_zh_usas_tagger,
)

# 打印标注结果
print("\n=== LLM USAS 中文语义标注结果预览 ===")
display_anno(llm_zh_usas_annos[0])

Semantic Tagging: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:00<?, ?it/s]


=== LLM USAS 中文语义标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'Z2'), ('近', 'N3.3-'), ('海滨', 'W3'), ('的', 'Z5'), ('一条', 'N1'), ('大路', 'M3'), ('上', 'M6'), ('，', 'PUNCT'), ('一队', 'N1'), ('清兵', 'G3'), ('手执', 'A1.1.1'), ('刀枪', 'G3'), ('，', 'PUNCT'), ('押着', 'S7.1+'), ('七辆', 'N1'), ('囚车', 'G2.1-'), ('，', 'PUNCT'), ('冲风冒寒', 'W4'), ('，', 'PUNCT'), ('向北', 'M6'), ('而', 'Z5'), ('行', 'M1'), ('。', 'PUNCT')]


#### **4. 中文语义标注模型对比：LLM & PyMUSAS**

In [5]:
# === 加载模型：PyUSAS USAS 赋码集 ===

# 语种：汉语
lang = 'chinese'

# 赋码集：USAS
# https://ucrel.lancs.ac.uk/usas/
# https://ucrel.lancs.ac.uk/usas/usas_guide.pdf
tagset = 'usas'

# 标注模式：pyMUSAS 本地模型
mode = 'local'

# PyMUSAS 模型：Rule-based Tagger
# https://ucrel.github.io/pymusas/usage/how_to/tag_text_with/rule_based_tagger
# https://github.com/UCREL/Multilingual-USAS/tree/master/Chinese
# 首次加载需从网络将模型下载至本地
t0 = time.time()
pymusas_zh_usas_tagger = SEMTagger(
    lang=lang,
    tagset=tagset,
    mode=mode,
)
t1 = time.time()
print('PyMUSAS USAS 中文语义标注标注模型加载完毕！')
print(f'加载耗时：{t1-t0:.2f}s')

PyMUSAS USAS 中文语义标注标注模型加载完毕！
加载耗时：12.27s


In [6]:
# === 标注原文：PyUSAS USAS 赋码集 ===

pymusas_zh_usas_annos = annotate_data(
    data['source'],
    pymusas_zh_usas_tagger,
)

# 打印标注结果
print("\n=== PyMUSAS USAS 中文语义标注标注结果预览 ===")
display_anno(pymusas_zh_usas_annos[0])

Semantic Tagging: 100%|████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 24.27it/s]


=== PyMUSAS USAS 中文语义标注标注结果预览 ===

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
[('江南', 'Z99'), ('近', 'A13.4'), ('海滨', 'M4'), ('的', 'Z5'), ('一', 'N1'), ('条', 'G2.1/P1'), ('大', 'N3.2+'), ('路上', 'Z99'), ('，', 'PUNCT'), ('一', 'N1'), ('队', 'A1.1.1'), ('清兵', 'Z99'), ('手', 'B1'), ('执', 'A9+'), ('刀枪', 'Z99'), ('，', 'PUNCT'), ('押', 'Z99'), ('着', 'Z2'), ('七', 'N1'), ('辆', 'N1'), ('囚车', 'M3'), ('，', 'PUNCT'), ('冲', 'A2.1+'), ('风', 'A1.1.1'), ('冒寒', 'Z99'), ('，', 'PUNCT'), ('向', 'Z5'), ('北', 'M6'), ('而', 'Z5'), ('行', 'A1.1.1'), ('。', 'PUNCT')]


In [7]:
# === 模型对比：LLM USAS & PyMUSAS USAS ===

print('中文语义标注模型对比（USAS 赋码集）：LLM & PyMUSAS')
compare_annos(
    llm_zh_usas_annos,
    pymusas_zh_usas_annos,
    annos_1_name="LLM",
    annos_2_name="PyMUAS",
)

中文语义标注模型对比（USAS 赋码集）：LLM & PyMUSAS

[ID]: 00001
江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
--------------------------------------------------------------------------------
Jaccard: 0.093
LLM: {('押着', 'S7.1+'), ('大路', 'M3'), ('一条', 'N1'), ('向北', 'M6'), ('囚车', 'G2.1-'), ('近', 'N3.3-'), ('行', 'M1'), ('清兵', 'G3'), ('冲风冒寒', 'W4'), ('海滨', 'W3'), ('上', 'M6'), ('刀枪', 'G3'), ('手执', 'A1.1.1'), ('七辆', 'N1'), ('江南', 'Z2'), ('一队', 'N1')}
PyMUAS: {('囚车', 'M3'), ('七', 'N1'), ('清兵', 'Z99'), ('行', 'A1.1.1'), ('向', 'Z5'), ('冒寒', 'Z99'), ('风', 'A1.1.1'), ('一', 'N1'), ('近', 'A13.4'), ('大', 'N3.2+'), ('冲', 'A2.1+'), ('海滨', 'M4'), ('辆', 'N1'), ('路上', 'Z99'), ('押', 'Z99'), ('手', 'B1'), ('条', 'G2.1/P1'), ('刀枪', 'Z99'), ('江南', 'Z99'), ('队', 'A1.1.1'), ('北', 'M6'), ('执', 'A9+'), ('着', 'Z2')}

[ID]: 00002
前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。
--------------------------------------------------------------------------------
Jaccard: 0.158
LLM: {('监禁', 'G2.1-'), ('一个', 'N1'), ('男子', 'S2.2'), ('囚车', 'G2.1-'), ('老者', 